In [1]:
from accelerate import notebook_launcher

from core.utils import *
from train import *

os.environ['WANDB_NOTEBOOK_NAME'] = str(pathlib.Path().absolute())
from accelerate.utils import write_basic_config

write_basic_config(mixed_precision="no", save_location='~/.cache/huggingface/accelerate/default_config.yaml')
config = TrainingConfig()

init(False)
print(accelerate.accelerator.AcceleratorState._shared_state)


def launch(x, args):
    os.environ["NCCL_P2P_DISABLE"] = "1"
    os.environ["NCCL_IB_DISABLE"] = "1"
    notebook_launcher(x, args=args, num_processes=2)

Configuration already exists at ~/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


wandb: Currently logged in as: mickjagger19. Use `wandb login --relogin` to force relogin
wandb: WARNING Disabling SSL verification.  Connections to this server are not verified and may be insecure!


{}


## Train VAE

In [2]:
vae_config = TrainingConfig()
vae_config.train_batch_size = 11
vae_config.num_epochs = 10
vae_config.save_model_epochs = 4
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
launch(
    train_vae, (vae_config, True, None))

Launching training on 2 GPUs.


wandb: WARNING Disabling SSL verification.  Connections to this server are not verified and may be insecure!


Epoch 9: 100%|██████████| 46/46 [01:13<00:00,  1.47it/s, loss=1.76e+3, lr=1.47e-7, step=459]

num_params=1009
num_params=1009

Epoch 9: 100%|██████████| 46/46 [01:14<00:00,  1.62s/it, loss=1.76e+3, lr=1.47e-7, step=459]


## Train Diffusion

In [3]:
diffusion_config = TrainingConfig()
diffusion_config.train_batch_size = 1
launch(train_diffusion, (diffusion_config,))

Launching training on one GPU.


NameError: name 'wandb' is not defined

## Test

In [2]:
config = TrainingConfig()
from test_model import *
import gc

# wandb.init()
S, img = generate_from_genre('jazz',
                             vae_path='/root/autodl-tmp/remixer/training/artifacts/AutoencoderKL:v1/model_epoch_0.pth',
                             generator_path='/root/autodl-tmp/remixer/training/artifacts/VQGenerator:v1/model_epoch_0.pth')

make_grid([img], 1)
gc.collect()

loading /root/autodl-tmp/remixer/training/artifacts/VQGenerator:v1/model_epoch_0.pth
loading /root/autodl-tmp/remixer/training/artifacts/AutoencoderKL:v1/model_epoch_0.pth


  0%|          | 0/1000 [00:00<?, ?it/s]

True


ParameterError: Audio buffer is not finite everywhere